In [1]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, BatchNormalization
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.regularizers import l2
import zipfile
import os

In [2]:
tf.random.set_seed(123)
tf.keras.utils.set_random_seed(123)
tf.config.experimental.enable_op_determinism()

In [3]:
# Prepare datasets
with zipfile.ZipFile('homer_bart.zip', 'r') as zip_ref:
    zip_ref.extractall('')

def read_image(file_path):
    file_bytes = tf.io.read_file(file_path)
    image = tf.image.resize(tf.image.decode_image(file_bytes, channels = 3), [64, 64])/255.0
    return image

homer_files = [os.path.join('Homer', file) for file in os.listdir('Homer') if file.endswith('bmp')]
bart_files = [os.path.join('Bart', file) for file in os.listdir('Bart') if file.endswith('bmp')]
homer_images = [read_image(image_file) for image_file in homer_files]
bart_images = [read_image(image_file) for image_file in bart_files]

x_train = homer_images[: (len(homer_images) - len(homer_images) // 9)] + bart_images[: (len(bart_images) - len(bart_images) // 9)]
y_train = [0] * (len(homer_images) - len(homer_images) // 9) + [1] * (len(bart_images) - len(bart_images) // 9)

x_test = homer_images[(len(homer_images) - len(homer_images) // 9) :] + bart_images[(len(bart_images) - len(bart_images) // 9) :]
y_test = [0] * (len(homer_images) // 9) + [1] * (len(bart_images) // 9)

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

batch_size = 32
train_dataset = train_dataset.shuffle(buffer_size = len(x_train)).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

In [4]:
model = Sequential(
    [
        tf.keras.Input(shape = (64, 64, 3)),
        Flatten(),
        BatchNormalization(),
        Dense(units = 64, activation = 'relu', kernel_regularizer=l2(0.1)),
        BatchNormalization(),
        Dense(units = 128, activation = 'relu'),
        Dense(units = 2, activation = 'softmax'),
    ]
)
model.compile(
    loss = SparseCategoricalCrossentropy(),
    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.00001),
    metrics = ['accuracy'],
)
model.fit(train_dataset, epochs = 100)

Epoch 1/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.5231 - loss: 13.5203
Epoch 2/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6035 - loss: 13.3570
Epoch 3/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7062 - loss: 13.2655
Epoch 4/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6837 - loss: 13.1941
Epoch 5/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7386 - loss: 13.0871
Epoch 6/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7946 - loss: 12.9928
Epoch 7/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8286 - loss: 12.9180
Epoch 8/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8054 - loss: 12.8426
Epoch 9/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8789 - loss: 12.7505
Epoch 10/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8822 - loss: 12.6762
Epoch 11/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8697 - loss: 12.6355
Epoch 12/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0

In [5]:
loss, accuracy = model.evaluate(test_dataset)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - accuracy: 0.8966 - loss: 7.7568
